## The Bilayer Builder: Creating Customizable Initial Configurations of Lipid Bilayers using mBuild

Welcome to the Bilayer Builder tutorial! In this lesson, you will learn how to use the aforementioned tool to create custom bilayer systems for molecular dynamics simulation.

**Note**: This is a tutorial meant to explain how to use a computational tool. It is not a lesson on lipid bilayers as a whole, and it assumes basic knowledge of these types of systems (terms like area per lipid and headgroup offset will be used assuming the user knows what they mean).

#### Lesson 1: Basic Syntax

Let's start off with some import statements to bring in mBuild, the Bilayer class itself, as well as the lipids we'll put into our bilayer.

In [4]:
import mbuild as mb
from mbuild.recipes.bilayer.bilayer import Bilayer
#from mbuild.lib.prototypes import DSPC

Great, now let's just create a simple bilayer, and then start digging into how this tool works.

In [5]:
bi = Bilayer(lipids=[(DSPC(), 1.0, 0, 0)])

bi.visualize()

/home/loganguy/builds/mbuild/mbuild/compound.py:1241: UserWarning: Performing energy minimization using the Open Babel package. Please refer to the documentation to find the appropriate citations for Open Babel and the UFF force field
  "Open Babel and the {} force field".format(forcefield))
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/pandas/io/parsers.py:2108: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/pandas/io/parsers.py:2110: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate_around_y is deprecated. Please use rotate(compound, theta, around=np.asarray([0, 1, 0]))
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate is deprecated. Please use Compound

Okay, a couple of things to note immediately:

* Time: the bilayer builder has to make a lot of calculations in order to do its job, so making these systems takes a while. However, this is merely a tutorial, so in the future we'll use molecules with alkyl hydrogens removed, and we'll simulate smaller systems
* The `lipids` parameter: that is the single required parameter (for now), and will be the subject of our next discussion

More importantly, let's briefly discuss how these bilayers are constructed:
* Lipid placement
    * The Bilayer Builder utilizes mBuild's `2DGridPattern` class, which creates a scaffold of positions that the lipids are placed onto. The size of this grid is controlled by the user.
* Solvation
    * Lipid bilayers do not exist in a vacuum; therefore, the Bilayer Builder has built-in capability to solvate the bilayer in water, which the user can control

The Bilayer Builder's strength is the control it gives to the user in determining physical properties of their specific lipid bilayer system. Therefore, there are numerous parameters which the Bilayer Builder requires (most are set to intelligent default values, and you would do good to minimize altering those defaults; however, this tutorial will explain how to control those variables). Let's begin with how to specify your desired composition.

#### Lesson 2: Controlling the Composition

*Parameters*:
1. `lipids`: This parameter specifies the composition of your bilayer, as well as a number of other lipid-specific parameters. This is the most involved parameter, and therefore merits more discussion
    * Data Type: `list of tuples`, where each tuple represents a single lipid molecule
    * Each tuple has four elements that must be given in this order:
        * The lipid itself; an `mb.Compound`
        * The fraction of the bilayer made up of that lipid; type `float`, must be between 0 and 1
        * Headgroup Offset: some lipid head groups tend to sink deeper into the bilayer when simulated alongside a phospholipid, so the Bilayer Builder tries to accomodate this by adjusting their starting position with respect to the bilayer normal with this variable (distance in nanometers)
            * Data Type: float; greater than zero moves the lipid "up" (into the solvent), less than zero move the lipid deeper into the bilayer
        * Reference atom: Each lipid is an mBuild `Compound`, and therefore is made up of `Particles`, all of which have an integer index. The reference atom is the atom which the user considers to be the top of the "head group" of the lipid.
            * This atom will form the bilayer-solvent interface; it is therefore necessary that the user know the components of any lipid they input into the Bilayer Builder
            
*Examples*:

A `lipids` parameter for a pure DSPC bilayer:
```
lipids = [(DSPC, 1.0, 0, 0)]
```

Create a bilayer composed of 50% DSPC and 50% C-16 alcohol (remember, we want to use united-atom molecules from now on to save time, and want to keep our bilayers to a reasonable size, so 2x2 should be fine).
**Help**: Use the list provided below for your `lipids` parameter. We've gotten you started with your imports and Bilayer instantiation, just add the lipids parameter.
```
lipids = [(DPSCUA(), 0.5, 0, 0), (ALCUA(16), 0.5, -0.4, 17)] 
```


In [9]:
from mbuild.lib.UA_molecules import *

bilayer = Bilayer(lipids = [(DSPCUA(), 0.5, 0, 0), (ALCUA(16), 0.5, -0.4, 17)], n_lipids_x=2, n_lipids_y=2, solvent_per_lipid=0)
bilayer.visualize()

/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate_around_y is deprecated. Please use rotate(compound, theta, around=np.asarray([0, 1, 0]))
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate is deprecated. Please use Compound.rotate()
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate_around_y is deprecated. Please use rotate(compound, theta, around=np.asarray([0, 1, 0]))
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate is deprecated. Please use Compound.rotate()
  warn(printed_message, DeprecationWarning)
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The install

Hopefully you observe a couple of things:
* There is no solvent because that is what was specified by the `solvent_per_lipid` parameter
* There are two of each kind of lipid in each leaflet, because we specified a 2x2 bilayer with the following new parameters
    3. `n_lipids_x`: The number of lipids in the x-dimension. Must be an integer. Controls the number of grid points along with n_lipids_y
    4. `n_lipids_y`: The number of lipids in the y-dimension. Must be an integer.
    
This worked out nicely because we specified fractions that made sense for the size we gave. As you use this tool, be sure to watch out for the following mistakes:

In [10]:
lipids = [(DSPCUA(), 0.33, 0, 0), (ALCUA(16), 0.33, -0.4, 17), (FFAUA(16, ester=False), 0.33, -0.4, 17)]
bilayer = Bilayer(lipids, n_lipids_x=2, n_lipids_y=2, solvent_per_lipid=0)
bilayer.visualize()

/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate_around_y is deprecated. Please use rotate(compound, theta, around=np.asarray([0, 1, 0]))
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate is deprecated. Please use Compound.rotate()
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate_around_y is deprecated. Please use rotate(compound, theta, around=np.asarray([0, 1, 0]))
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate is deprecated. Please use Compound.rotate()
  warn(printed_message, DeprecationWarning)


ValueError: Lipid fractions do not add up to 1.

Whoops! If you are going to specify more than one component, make sure that all of the fractions add up to 1.0; otherwise, your composition is non-sensical.

Let's fix that problem by changing one of the 0.33 to 0.34.

In [11]:
lipids = [(DSPCUA(), 0.33, 0, 0), (ALCUA(16), 0.33, -0.4, 17), (FFAUA(16, ester=False), 0.34, -0.4, 17)]
bilayer = Bilayer(lipids, n_lipids_x=2, n_lipids_y=2, solvent_per_lipid=0)
bilayer.visualize()

/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate_around_y is deprecated. Please use rotate(compound, theta, around=np.asarray([0, 1, 0]))
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate is deprecated. Please use Compound.rotate()
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate_around_y is deprecated. Please use rotate(compound, theta, around=np.asarray([0, 1, 0]))
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate is deprecated. Please use Compound.rotate()
  warn(printed_message, DeprecationWarning)
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The install

The Bilayer Builder was able to create a system for you, but look closely. Each leaflet has one DSPC molecule, one alcohol molecule, and **two** free fatty acid molecules.

Presumably, this is not what we would want when we specify fractions of 0.33, 0.33, and 0.34. However, we specified a size of 2x2, and the Bilayer Builder in its current state does not tell you when the actual fraction deviates from your expected fraction.

**IMPORTANT**: Always take care that your combination of size and composition specifications make sense together.

#### Lesson 3: Controlling the area per lipid

With a single parameter, we can drastically alter the area of the bilayer.
1. `area_per_lipid`: in units of square nanometers (for an APL of 53 square Angstroms, `area_per_lipid=0.53`)
    * NOTE: changing the APL is for educational purposes ONLY in this tutorial. Normally, the Bilayer Builder intelligently sets the APL based on bilayer composition.
    * It is not recommended that you change this APL default, as it could cause simulation problems down the road
Let's use the same lipids as above, and make the bilayer much more spaced.

In [13]:
bilayer = Bilayer(lipids, n_lipids_x=3, n_lipids_y=3, area_per_lipid=1.0, solvent_per_lipid=0)
bilayer.visualize()

/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.


Now, change the `area_per_lipid` parameter in the cell above to 0.3, and see the bilayer condense.

#### Lesson 4: Changing the geometry of the lipids

So far, we've only explored parameters that alter properties of the overall bilayer system (solvation, dimensions, composition, area per lipid). Let's explore the different parameters that we can use to change the geometric orientations of the lipids themselves.
1. `tilt_angle`: float, default is a random angle between 25 and 35 degrees
    * Here, the lipids are each tilted with respect to the y-axis by the angle produced by the random number generator. Each lipid will have the same tilt angle
    
Let's first make a bilayer with no tilt angle, using the same parameters as above.

In [15]:
bilayer = Bilayer(lipids, n_lipids_x=3, n_lipids_y=3, area_per_lipid=1.0, 
                  tilt_angle=0, solvent_per_lipid=0)
bilayer.visualize()

/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.


You can see that the lipids are parallel with the bilayer normal. Try rerunning the cell above after changing the `tilt_angle` parameter above (something around 30 is normal).

Controlling this parameter is useful if you want to create an initial configuration that is closer to a specific final configuration.

You may notice that when you add a tilt angle to the bilayer, the tail groups don't actually line up with each other like a simulated bilayer. That's because, in an effort to add some randomization to the initial configuration, the Bilayer Builder randomly rotates each lipid with respect to the z-axis when it is placed onto the `Pattern`, after the tilt angle is applied. The specific parameter used to accomplish this is given:
2. `max_tail_randomization`: the maximum amount each lipid can be randomly rotated with respect to the z-axis in degrees (to minimize steric overlap resulting from the rotation)
    * Data Type: float
    * Default: 25

Let's see what our bilayer looks like when we eliminate this random rotation (as usual, we will keep the same parameters from above):

In [17]:
bilayer = Bilayer(lipids, n_lipids_x=3, n_lipids_y=3, area_per_lipid=1.0, 
                  tilt_angle=30, max_tail_randomization=0, solvent_per_lipid=0)
bilayer.visualize()

/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.


Changing the `max_tail_randomization` parameter above will add the randomization back into the bilayer. It is not recommended that you change this parameter; most simulations require somewhat randomized initial configurations in order to prove validity of the final configuration.

Finally, if you look very closely at the bilayer visualized above, you'll notice that some of the head groups sit deeper in the bilayer than others. This is intended to make the initial configurations closer to reality. Changing this is a bit more involved:

*Altering the head group offset distance*

Distinct lipid molecules have distinct headgroup offsets; therefore, in order to change them, you'll have to alter the way you construct your `lipids` parameter. Go back to Lesson 2 and reread the individual elements contained in the `lipids` parameter...

Changing the third element of a single lipid tuple will change the head group offset for that lipid. Observe for a DSPC/FFA16 system below:

In [18]:
lipids = [(DSPCUA(), 0.5, 0, 0), (FFAUA(16, ester=False), 0.5, -0.4, 17)]
bilayer = Bilayer(lipids, n_lipids_x=2, n_lipids_y=2, area_per_lipid=1.0, 
                  tilt_angle=30, max_tail_randomization=25, solvent_per_lipid=0)
bilayer.visualize()

/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate_around_y is deprecated. Please use rotate(compound, theta, around=np.asarray([0, 1, 0]))
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate is deprecated. Please use Compound.rotate()
  warn(printed_message, DeprecationWarning)
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.


See how the free fatty acid head groups sit lower than those of DSPC? Change the third element of the FFA16 tuple to 0, and observe the change.

#### Lesson 5: Other Bilayer Properties

There are a number of other features that the Bilayer Builder allows you to customize. Some of those features are explored below:

1. `cross_tilt`: some bilayers display an interesting cross tilt property. Setting this parameter to `True` creates that phenomenon
2. `solvent`: the `mb.Compound` that the bilayer will be solvated with, defaulted to water
3. `solvent_per_lipid`: defaulted to 20, number of water molecules in the solvent phase per lipid molecule in the bilayers

Run the cells below to see the difference between a normal bilayer and a cross-tilted bilayer.

In [1]:
import mbuild as mb
from mbuild.recipes.bilayer.bilayer import Bilayer
from mbuild.lib.UA_molecules import *

lipids = [(DSPCUA(), 0.5, 0, 0), (FFAUA(16, ester=False), 0.5, -0.4, 17)]
bilayer = Bilayer(lipids, n_lipids_x=4, n_lipids_y=4, area_per_lipid=1.0, 
                  tilt_angle=30, max_tail_randomization=25, solvent_per_lipid=0, 
                  cross_tilt=True)
bilayer.visualize()

/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate_around_y is deprecated. Please use rotate(compound, theta, around=np.asarray([0, 1, 0]))
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate is deprecated. Please use Compound.rotate()
  warn(printed_message, DeprecationWarning)
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.


In [4]:
bilayer = Bilayer(lipids, n_lipids_x=4, n_lipids_y=4, area_per_lipid=1.0, 
                  tilt_angle=30, max_tail_randomization=25, solvent_per_lipid=0, 
                  cross_tilt=True)
bilayer.visualize()

/home/loganguy/builds/mbuild/mbuild/compound.py:1746: UserWarning: Guessing that "<OW pos=( 1.0203, 0.7604, 1.2673), 0 bonds, id: 140028094757352>" is element: "O"
  warn('Guessing that "{}" is element: "{}"'.format(atom, element))
/home/loganguy/builds/mbuild/mbuild/compound.py:1746: UserWarning: Guessing that "<HW1 pos=( 0.9626, 0.8420, 1.2673), 0 bonds, id: 140028105938368>" is element: "H"
  warn('Guessing that "{}" is element: "{}"'.format(atom, element))
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.


Change the value of the parameter `solvent_per_lipid` in either of the above cells to 20 to see a solvated bilayer.

#### Conclusion and Using the Bilayer Builder via Command Line

We've seen a lot of input parameters that you can change to fit your needs. In reality, most of the variables have intelligent defaults, and don't need to be changed; most of the time, you will created a bilayer by simply doing the following:


In [2]:
bilayer = Bilayer(lipids)
bilayer.visualize()

/home/loganguy/builds/mbuild/mbuild/compound.py:1746: UserWarning: Guessing that "<OW pos=( 1.0203, 0.7604, 1.2673), 0 bonds, id: 140067500841448>" is element: "O"
  warn('Guessing that "{}" is element: "{}"'.format(atom, element))
/home/loganguy/builds/mbuild/mbuild/compound.py:1746: UserWarning: Guessing that "<HW1 pos=( 0.9626, 0.8420, 1.2673), 0 bonds, id: 140067511821760>" is element: "H"
  warn('Guessing that "{}" is element: "{}"'.format(atom, element))
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.


Now that you've completed this tutorial, you have a very good working knowledge of the main features of the Bilayer Builder. You can customize your own bilayer to your specifications.

**NOTE**: The Bilayer Builder does have restrictions on what types of molecules it will work with. For geometric requirements of input lipids, see the docstrings in the bilayer.py file.

***Command line usage***: In order to enable file creation, and in order to use automatically generated lipid parameters, the Bilayer Builder should be run as a script via command line. Using your terminal, enter the bilayer directory under lib inside mbuild, and enter the following:

```bash
python bilayer.py -h
```

This will lead you to the help menu, and will give an explanation of command line usage. For tips and common errors encountered while simulating the systems created by the Bilayer Builder, see the file inside the bilayer directory. Happy simulating!